In [ ]:
import numpy

In [ ]:
%cd ..
%run visualisation.ipynb
%run common.ipynb

#g = read_nba_full("../Data/Basketball/Players.csv",
#                  "../Data/Basketball/Edges.csv",
#                  False, True)

all11_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588, 2586,4198,115,2925,1459,2235,1794,415,1460,564,4168,2384,1142,3134,1849,2145,1945,3283,3669,1446,718,1031,1898,2577,2492,3502,620,3079,3672,1995,3383,4609,4090,65,2930,1504]
#nba11_20 = g.subgraph(all11_20)

all15_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588, 2586,4198,115,2925,1459,2235,1794,415,1460,564,4168,2384,1142]
#nba15_20 = g.subgraph(all15_20)

all19_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159]
#nba19_20 = g.subgraph(all19_20)

#github = read_github_full("../Data/Github/nodes_wzab.csv",
#                  "../Data/Github/edges_wzab.csv")

In [ ]:
def mod(graph, community):
    M_in = 0
    M_out = 0
    for node in community:
        for edge_ind in graph.incident(node):
            edge = graph.es[edge_ind]
            if node == edge.target_vertex and node == edge.source_vertex:
                continue
            elif node == edge.target_vertex:
                if edge.source_vertex in community:
                    M_in+=1
                else:
                    M_out+=1
            elif node  == edge.source_vertex:
                if edge.target_vertex in community:
                    M_in+=1
                else:
                    M_out+=1
    if M_out == 0:
        return len(graph.vs) + 1
    return M_in / 2 / M_out

In [ ]:
def mod_f(graph, community, alpha):
    M_in = 0
    M_out = 0
    for node in community:
        edges = graph.incident(node)
        for e in edges:
            edge = graph.es[e]
            if node == edge.target_vertex and node == edge.source_vertex:
                print("Warn - out vertex equals current vertex")
            elif node == edge.target_vertex:
                if edge.source_vertex in community:
                    M_in+=1
                else:
                    M_out+=1
            elif node == edge.source_vertex:
                if edge.target_vertex in community:
                    M_in+=1
                else:
                    M_out+=1
    if M_in == 0 and M_out == 0:
        return 0
    
    return M_in / pow(M_in + M_out, alpha)

In [ ]:
def step1(graph, other_modularity):
    print("basic ocdlce begin ...")
    communities = []
    for v in graph.vs:
        v["communities"] = []
    for u in graph.vs:
        for v in graph.neighbors(u):
            v = graph.vs[v]
            if len(set(u["communities"]).intersection(v["communities"])) == 0:
                tmp_comm = [u,v]
                NC = list(set(graph.vs[graph.neighbors(v)]).intersection(graph.vs[graph.neighbors(u)]))
                if len(NC) > 2:
                    for node in NC:
                        if other_modularity:
                            if  mod_f(graph, tmp_comm + [node], 0.5) > mod_f(graph, tmp_comm, 0.5):
                                tmp_comm = tmp_comm + [node]
                        else:                            
                            if mod(graph, tmp_comm + [node]) > mod(graph, tmp_comm):
                                tmp_comm = tmp_comm + [node]
                if len(tmp_comm) > 4:
                    id = len(communities)+1
                    communities.append({"nodes": tmp_comm, "id": id })
                    for node in tmp_comm:
                        node["communities"].append(id)

    return graph, communities


In [ ]:
def step1_modified(graph, other_modularity):
    print("modified ocdlce begin ...")
    communities = []
    for v in graph.vs:
        v["communities"] = []
    i=0
    for u in graph.vs:
        for v in graph.neighbors(u):
            v = graph.vs[v]
            if len(set(u["communities"]).intersection(v["communities"])) == 0:
                for w in graph.neighbors(u):
                    w = graph.vs[w]
                    if w!=v and len(set(w["communities"]).intersection(v["communities"])) == 0 and len(set(w["communities"]).intersection(u["communities"])) == 0:
                        tmp_comm = [u,v,w]
                        NC = list(set(graph.vs[graph.neighbors(v)]).intersection(graph.vs[graph.neighbors(u)]))
                        NC = list(set(NC).intersection(graph.vs[graph.neighbors(w)]))
                        if len(NC) > 2:
                            for node in NC:
                                if other_modularity:
                                    if mod_f(graph, tmp_comm + [node], 0.5) > mod_f(graph, tmp_comm,0.5):
                                        tmp_comm = tmp_comm + [node]
                                else:                            
                                    if mod(graph, tmp_comm + [node]) > mod(graph, tmp_comm):
                                        tmp_comm = tmp_comm + [node]
                        if len(tmp_comm) > 4:
                            id = len(communities)+1
                            communities.append({"nodes": tmp_comm, "id": id })
                            for node in tmp_comm:
                                node["communities"].append(id)
                            continue


    return graph, communities

In [ ]:
def E_in(graph, nodes):
    edges = []
    for node in nodes:
        inc_edges = graph.incident(node)
        for e in inc_edges:
            edge = graph.es[e]
            if node == edge.target_vertex and node == edge.source_vertex:
                print("Warn - out vertex equals current vertex")
            elif node == edge.target_vertex:
                if edge.source_vertex in nodes:
                    edges.append(edge)
            elif node == edge.source_vertex:
                if edge.target_vertex in nodes:
                    edges.append(edge)
    return edges

In [ ]:
def WOS(graph, com1, com2, alfa, beta):
    v = len(set(com1["nodes"]).intersection(com2["nodes"]))
    min_v = min(len(com1["nodes"]),len(com2["nodes"]))
    e1 = E_in(graph, com1["nodes"])
    e2 = E_in(graph, com2["nodes"])
    e = len(set(e1).intersection(e2))
    min_e = min(len(e1),len(e2))
    res = alfa * v / min_v + (1-alfa)*e/min_e >= beta
    return res

In [ ]:
def Union(graph, all_coms, com, alfa, beta):
    res = []
    for c in all_coms:
        if c["id"] != com["id"]:
            if WOS(graph, c, com, alfa, beta):
                res.append(c)
    return res

In [ ]:
def step2(graph, local_communities, alfa, beta):
    C = local_communities
    for com in C:
        res = Union(graph, C, com, alfa, beta)
        for c in res:
            for node in c["nodes"]:
                node["communities"].remove(c["id"])
                if com["id"] not in node["communities"]:
                    node["communities"].append(com["id"])
            C.remove(c)
    return graph, C

In [ ]:
def step3(graph, local_communities,  other_modularity):
    for v in graph.vs:
        if len(v["communities"]) == 0:
            coms = []
            for u in graph.neighbors(v):
                node = graph.vs[u]
                coms =  list(set(coms).union(node["communities"]))
            for com in coms:
                c = next(item for item in local_communities if item["id"] == com)
                if other_modularity:
                    if mod_f(graph, c["nodes"] + [v], 0.5) > mod_f(graph, c["nodes"], 0.5):
                        v["communities"].append(c["id"])
                        c["nodes"] = c["nodes"] + [v]
                else:                            
                    if mod(graph, c["nodes"] + [v]) > mod(graph, c["nodes"]):
                        v["communities"].append(c["id"])
                        c["nodes"] = c["nodes"] + [v]

    return graph


## Benchmarks

In [ ]:
def my_basic_ocdlce(graph):
    g, com = step1(graph, False)
    g, com = step2(g, com, 0.5, 0.5)
    g = step3(g,com, False)

In [ ]:
def my_modified_basic_ocdlce(graph):
    g, com = step1_modified(graph, False)
    g, com = step2(g, com, 0.5, 0.5)
    g = step3(g,com, False)

In [ ]:
def my_basic_ocdlce_with_modularity_f(graph):
    g, com = step1(graph, True)
    g, com = step2(g, com, 0.5, 0.5)
    g = step3(g,com,True)

In [ ]:
def my_modified_ocdlce_with_modularity_f(graph):
    g, com = step1_modified(graph, True)
    g, com = step2(g, com, 0.5, 0.5)
    g = step3(g,com,True)

In [ ]:
def perform_full_benchmark_ocdlce(max_vertices):
    perform_default_benchmark(my_basic_ocdlce, 'my-basic-ocdlce-benchmark.txt', max_vertices)
    perform_default_benchmark(my_basic_ocdlce, 'my-modified-ocdlce-benchmark.txt', max_vertices)
    perform_default_benchmark(my_basic_ocdlce, 'my-basic-ocdlce-with-modularity-f-benchmark.txt', max_vertices)
    perform_default_benchmark(my_basic_ocdlce, 'my-modified-ocdlce-with-modularity-f-benchmark.txt', max_vertices)
